## Installing Libraries

In [ ]:
!pip install transformers -U -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 75.7 MB/s eta 0:00:00


In [ ]:
! pip install sentencepiece

In [ ]:
!pip freeze | grep transformers

sentence-transformers==3.2.1
transformers==4.47.0


## importing Libraries

In [ ]:
import gradio as gr
from PIL import Image
import pytesseract
from transformers import MBartForConditionalGeneration, MBart50Tokenizer


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Loading model

In [ ]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

## transating image functions

In [ ]:
def translate_image(img_path, src_lang, tgt_lang):
        # Extract text from the image using Tesseract OCR
        text = pytesseract.image_to_string(img_path)
        if not text.strip():
            return "No text detected. Please upload a valid image.", ""

        # Translating the text
        tokenizer.src_lang = src_lang
        inputs = tokenizer(text, return_tensors="pt")
        generated_tokens = model.generate(
            **inputs, forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang]
        )
        translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]



## Putting it all together in the gradio webpage

In [ ]:
# Installing necessary libraries
!pip install gradio pytesseract transformers sentencepiece

# Installing Tesseract OCR
!apt-get install -y tesseract-ocr
!pip install Pillow

# Importing libraries
import gradio as gr
from PIL import Image
import pytesseract
from transformers import MBartForConditionalGeneration, MBart50Tokenizer

# Loading the MBart model and tokenizer
MODEL_NAME = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50Tokenizer.from_pretrained(MODEL_NAME)
model = MBartForConditionalGeneration.from_pretrained(MODEL_NAME)

# Defining the OCR and translation function
def translate_image(img_path, src_lang, tgt_lang):
    try:
        # Extract text from the image using Tesseract OCR
        text = pytesseract.image_to_string(img_path)
        if not text.strip():
            return "No text detected. Please upload a valid image.", ""

        # Translating the text
        tokenizer.src_lang = src_lang
        inputs = tokenizer(text, return_tensors="pt")
        generated_tokens = model.generate(
            **inputs, forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang]
        )
        translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

        return text, translated_text
    except Exception as e:
        return f"Error: {str(e)}", ""

# Defining the Gradio interface
with gr.Blocks() as app:
    gr.Markdown("## OCR and Translation App")
    gr.Markdown("Upload an image, specify source and target languages, and get the translated text.")

    with gr.Row():
        img_input = gr.File(label="Upload Image", type="filepath")
        src_lang = gr.Textbox(label="Source Language Code (OCR works best with english)")
        tgt_lang = gr.Textbox(label="Target Language Code (any target language)")

    with gr.Row():
        extracted_text = gr.Textbox(label="Extracted Text")
        translated_text = gr.Textbox(label="Translated Text")

    translate_button = gr.Button("Translate")
    translate_button.click(
        translate_image, inputs=[img_input, src_lang, tgt_lang], outputs=[extracted_text, translated_text]
    )

# Launching the app
app.launch()

# Language codes. Note - The OCR app can only translate from english to the languges mentioned below

# Arabic (ar_AR), Czech (cs_CZ), German (de_DE), English (en_XX), Spanish (es_XX),
# Estonian (et_EE), Finnish (fi_FI), French (fr_XX), Gujarati (gu_IN), Hindi (hi_IN),
# Italian (it_IT), Japanese (ja_XX), Kazakh (kk_KZ), Korean (ko_KR), Lithuanian (lt_LT),
# Latvian (lv_LV), Burmese (my_MM), Nepali (ne_NP), Dutch (nl_XX), Romanian (ro_RO),
# Russian (ru_RU), Sinhala (si_LK), Turkish (tr_TR), Vietnamese (vi_VN), Chinese (zh_CN),
# Afrikaans (af_ZA), Azerbaijani (az_AZ), Bengali (bn_IN), Persian (fa_IR), Hebrew (he_IL),
# Croatian (hr_HR), Indonesian (id_ID), Georgian (ka_GE),
# Khmer (km_KH), Macedonian (mk_MK), Malayalam (ml_IN), Mongolian (mn_MN), Marathi (mr_IN),
# Polish (pl_PL), Pashto (ps_AF), Portuguese ), Swedish (sv_SE), Swahili (sw_KE), Tamil (ta_IN),
# Telugu (te_IN), Thai (th_TH), Tagalog (tl_XX), Ukrainian (uk_UA), Urdu (ur_PK), Xhosa (xh_ZA),
# Galician (gl_ES), Slovene (sl_SI)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://08e78290e64bdebf00.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
